In [ ]:
# import packages
import numpy as np
from numpy import random
from scipy import stats
import pandas as pd
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.model_selection import train_test_split


In [ ]:
data = pd.read_csv('/Users/arberimbibaj/dataset_example_indicatorCATE.csv', header=None, index_col=[0])
data = data.to_numpy()
data

In [ ]:
# train test split
random.shuffle(data)
training, test = data[:700,:], data[700:,:]

In [ ]:
# slice dataset by treatment status
training_control = training[training[:,26]==0]
training_treatment = training[training[:,26]==1]

Y_train_control = training_control[:,0]
Y_train_treatment = training_treatment[:,0]

X_train_control = training_control[:,1:26]
X_train_treatment = training_treatment[:,1:26]

X_test = test[:,1:26]
Y_test = test[:,0]

X_train = training[:,1:26]
Y_train = training[:,0]

W_train = training[:,26]
W_test = test[:,26]

tau_test = test[:,27]

In [ ]:
X_train_control

In [ ]:
X_train_treatment

In [ ]:
Y_train_control

In [ ]:
Y_train_treatment

In [ ]:
X_train

In [ ]:
W_train

In [ ]:
Y_train

In [ ]:
X_test

In [ ]:
W_test

In [ ]:
Y_test

In [ ]:
tau_test

# T-Learner

In [ ]:
# T-Learner (example with Random Forest)

# mu_0
t_learner_mu0 = RandomForestRegressor(max_depth=100, random_state=0)
t_learner_mu0.fit(X_train_control,Y_train_control)
t_mu_0_hat = t_learner_mu0.predict(X_test)

# mu_1
t_learner_mu1 = RandomForestRegressor(max_depth=100, random_state=0)
t_learner_mu1.fit(X_train_treatment,Y_train_treatment)
t_mu_1_hat = t_learner_mu1.predict(X_test)
# Prediction = mu_1 - mu_0
t_tau_hat = t_mu_1_hat - t_mu_0_hat
print(t_tau_hat)

In [ ]:
# mean squared error
((t_tau_hat - tau_test)**2).mean()

# S-Learner

In [ ]:
### S-Learner

In [ ]:
# set training and test features for the S-Learner (it views W as no different from other X's)
X_W_train = training[:,1:27]
X_test_0 = np.concatenate((test[:,1:26],np.zeros((300,1))), axis=1)
X_test_1 = np.concatenate((test[:,1:26],np.ones((300,1))), axis=1)
Y_train = training[:,0]

In [ ]:
X_test_0

In [ ]:
X_test_1

In [ ]:
Y_train

In [ ]:
# S-learner (example with Random Forest)

# mu_x
s_learner = RandomForestRegressor(max_depth=100, random_state=0)
s_learner.fit(X_W_train,Y_train)

# mu_0_hat
s_mu_0_hat = s_learner.predict(X_test_0)

# mu_1_hat
s_mu_1_hat = s_learner.predict(X_test_1)

# tau_hat
s_tau_hat = s_mu_1_hat - s_mu_0_hat
s_tau_hat


In [ ]:
# mean squared error
((s_tau_hat - tau_test)**2).mean()

# X-Learner

In [ ]:
### X-Learner

# mu_0 (same procedure as for t-learner, maybe can speed up process)
x_learner_mu0 = RandomForestRegressor(max_depth=100, random_state=0)
x_learner_mu0.fit(X_train_control,Y_train_control)

# mu_1 (same procedure as for t-learner, maybe can speed up process)
x_learner_mu1 = RandomForestRegressor(max_depth=100, random_state=0)
x_learner_mu1.fit(X_train_treatment,Y_train_treatment)

# compute imputed treatment effect D_0 and D_1
# d_0
imputed_0 = x_learner_mu1.predict(X_train_control) - Y_train_control

# d_1
imputed_1 = Y_train_treatment - x_learner_mu0.predict(X_train_treatment)

# regress imputed on X
# tau_hat_0
x_tau_0_hat = RandomForestRegressor(max_depth=100, random_state=0)
x_tau_0_hat.fit(X_train_control ,imputed_0)

# tau_hat_1
x_tau_1_hat = RandomForestRegressor(max_depth=100, random_state=0)
x_tau_1_hat.fit(X_train_treatment ,imputed_1)

# estimate e_x to use as g_x
g_x_hat = RandomForestClassifier(max_depth=100, random_state=0)
g_x_hat.fit(X_train,W_train)
probabilities = g_x_hat.predict_proba(X_test)
probas_1 = probabilities[:,1]
probas_0 = probabilities[:,0]

# final estimator of tau
x_tau_hat = probas_1 * x_tau_0_hat.predict(X_test) + probas_0 * x_tau_1_hat.predict(X_test)
x_tau_hat


In [ ]:
# mean squared error (much lower here!)
((x_tau_hat - tau_test)**2).mean()

# R-Learner

In [ ]:
### R-Learner


In [ ]:
### DR-Learner

In [ ]:
### RA-Learner

In [ ]:
### PW-Learner

In [ ]:
### F-Learner

In [ ]:
### U-Learner